In [1]:
!pip install datasets
from datasets import load_dataset
import json
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

FOLDER_PATH = "/content/drive/MyDrive/reasoning_multimodal_LLMs/example_data"
def split_convert_mathvision_to_json(split):
    # Load the dataset
    ds = load_dataset("MathLLMs/MathVision")

    # Combine train and test splits for reshuffling
    train_data = []
    test_data = []

    test_mini_ids = ds['testmini']['id']
    for item in ds[split]:
        if item['id'] in test_mini_ids:
            test_data.append(item)
        else:
            train_data.append(item)

    # Shuffle and split the data (80% train, 20% test)
    # train_data, test_data = train_test_split(all_data, test_size=0.2, random_state=42)

    # Process each split
    train_json = process_split(train_data)
    test_json = process_split(test_data)

    # Save to files
    with open(f"{FOLDER_PATH}/mathvision_train.json", 'w') as f:
        json.dump(train_json, f, indent=2)

    with open(f"{FOLDER_PATH}/mathvision_test.json", 'w') as f:
        json.dump(test_json, f, indent=2)

    print(f"Converted {len(train_json)} entries for train split")
    print(f"Converted {len(test_json)} entries for test split")

    return train_json, test_json

def convert_mathvision_to_json(split):
    # Load the dataset
    ds = load_dataset("MathLLMs/MathVision")
    data_json = process_split(ds[split])

    # Save to files
    with open(f"{FOLDER_PATH}/mathvision_{split}.json", 'w') as f:
        json.dump(data_json, f, indent=2)

    print(f"Converted {len(data_json)} entries for test split")

    return data_json

def process_split(data):
    converted_data = []
    for item in tqdm(data):
        question_prompt = item['question']
        if item.get('options') and len(item['options']) > 0:
            question_prompt += f" The answer is one of the options {', '.join(item['options'])}"

        conversation_entry = {
            "system_prompt": "Solve the following math problem step by step, given the image attached. Write the final answer after <Answer>",
            "image": item.get("image"),
            "conversations": [
                {
                    "from": "human",
                    "value": f"<image>{question_prompt}"
                },
                {
                    "from": "gpt",
                    "value": item.get('answer', 'No answer available.')
                }
            ]
        }

        converted_data.append(conversation_entry)

    return converted_data

# Run the conversion
train_json, test_json = split_convert_mathvision_to_json('test')

# Convert testmini split
testmini_json = convert_mathvision_to_json('testmini')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.2 MB/s eta 0:00:00
Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.93k [00:00<?, ?B/s]

(…)-00000-of-00001-3532b8d3f1b4047a.parquet:   0%|          | 0.00/57.0M [00:00<?, ?B/s]

(…)-00000-of-00001-f8ff70fcb2f29b1d.parquet:   0%|          | 0.00/6.99M [00:00<?, ?B/s]

...

Generating testmini split:   0%|          | 0/304 [00:00<?, ? examples/s]

100%|██████████| 304/304 [00:00<00:00, 296734.56it/s]


Converted 2736 entries for train split
Converted 304 entries for test split


100%|██████████| 304/304 [00:00<00:00, 438.59it/s]


Converted 304 entries for test split
